In [5]:
# Restart kernel to reload preprocess.py with augmentation support
import sys
sys.path.append("../..")

# Force reload the module to get latest changes
import importlib
import lunar_crater_age_logic.preprocess as preprocess_module
importlib.reload(preprocess_module)
from lunar_crater_age_logic.preprocess import load_data

from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

### 1. Data Loading

In [2]:
# Configuration
DATA_DIR = Path("/home/santanu/code/VMontejo/lunar-crater-age-classifier/raw_data/train")
IMG_HEIGHT = 227  # Changed to match preprocess.py
IMG_WIDTH = 227   # Changed to match preprocess.py
BATCH_SIZE = 32
#EPOCHS = 50
NUM_CLASSES = 3

In [3]:
# Load data using your preprocess.py function
print("Loading data from preprocess.py...")
train_ds, val_ds, test_ds, train_count, val_count, test_count = load_data(
    data_dir=DATA_DIR.parent,
    model_type='custom',
    normalization='zscore',
    batch_size=32,
    seed=42,
    augment_train=True,  # Enable TensorFlow augmentation (rotation, flip, brightness, contrast, zoom)
    train_balanced=True,
    train_weighted_sampling=False
)

class_names = ["ejecta", "oldcrater", "none"]
print(f"Class names: {class_names}")

Loading data from preprocess.py...
Loading data for custom
Normalization: zscore
Batch size: 32
Training: TensorFlow augmentation ENABLED (rotation, flip, brightness, contrast, zoom)
Training: BALANCED (358 per class)
Balanced train: 1074 images (358 per class)


2025-12-15 10:59:44.905095: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)



Data loaded:
Training: 1074 images (33 batches)
Validation: 613 images (19 batches)
Test: 779 images (24 batches)
Class names: ['ejecta', 'oldcrater', 'none']


### 2. Unbatch and distribute class weight same

In [ ]:
# Unbatch the training dataset to facilitate filtering
train_unbatched = train_ds.unbatch()


# Split train_ds based on class labels

ejecta_ds = train_unbatched.filter(lambda x, y: tf.equal(y, 0))
old_ds    = train_unbatched.filter(lambda x, y: tf.equal(y, 1))
none_ds   = train_unbatched.filter(lambda x, y: tf.equal(y, 2))

# Repeat each dataset indefinitely to allow sampling
ejecta_ds = ejecta_ds.repeat()
old_ds    = old_ds.repeat()
none_ds   = none_ds.repeat()

# make a balanced dataset by sampling equally from each class
balanced_train_ds = tf.data.experimental.sample_from_datasets(
    [ejecta_ds, old_ds, none_ds],
    weights=[0.33, 0.33, 0.34]
)

# Batching and prefetching
BATCH_SIZE = 32

# Final balanced, shuffled, batched, and prefetched training dataset
balanced_train_ds = (
    balanced_train_ds
        .shuffle(2000)
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
)


Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.


### 3. Custom model

In [ ]:
#A. Model structure
# --- 1. My FAVORITE OLD MODEL ARCHITECTURE ---

def build_lroc_model(input_shape, num_classes):
    model = models.Sequential(name="LROC_Custom_CNN_RGB")

    model.add(layers.InputLayer(shape=input_shape))

    # --- Block 1 --- Edge & Lines ---
    model.add(layers.Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 2 --- Simple Shape ---
    model.add(layers.Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 3 --- Complex texture ---
    model.add(layers.Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 4 --- Deeper Features ---
    model.add(layers.Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Block 5 --- Deepest Textures (Crucial for Old Craters) ---
    model.add(layers.Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2,2)))

    # --- Classification ---
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(256, kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.5))

    # Output layer must be softmax
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [7]:
# --- 2. THE FIX: ONE-HOT ENCODING HELPER ---
# Label Smoothing REQUIRES One-Hot labels (e.g., [0, 1, 0]).
# dataset currently gives Integers (e.g., 1). This function converts them.
def one_hot_encode(image, label):
    # num_classes = 3
    label = tf.one_hot(label, 3)
    return image, label

# --- 3. EXECUTION BLOCK () ---

# A. Convert your datasets to One-Hot
# (Assuming train_ds and val_ds are already loaded via load_data)
train_ds_smooth_1 = balanced_train_ds.map(one_hot_encode, num_parallel_calls=tf.data.AUTOTUNE)
val_ds_smooth   = val_ds.map(one_hot_encode, num_parallel_calls=tf.data.AUTOTUNE)

In [8]:
# B. Build Model
model = build_lroc_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
model.summary()

Model: "LROC_Custom_CNN_RGB"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 227, 227, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 227, 227, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 227, 227, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 113, 113, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 113, 113, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 113, 113, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 14, 14, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 14, 14, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,32

 Total params: 1,704,643 (6.50 MB)

 Trainable params: 1,702,659 (6.50 MB)

 Non-trainable params: 1,984 (7.75 KB)

In [10]:
# C. Compile with Label Smoothing
# This replaces Class Weights. It improves score without instability.
# Add Cosine LR Schedule
import math
initial_lr = 0.0001
first_decay_steps =900 #first_decay_steps = steps_per_epoch * 3

cosine_lr = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=initial_lr,
    first_decay_steps=first_decay_steps,
    t_mul=2.0,
    m_mul=1.0,
    alpha=1e-6   # minimum LR
)

optimizer = optimizers.AdamW(
    learning_rate=cosine_lr,
    weight_decay=1e-4, # Adding slight decay helps AdamW work better
    global_clipnorm=1.0,
)

model.compile(
    optimizer=optimizer,
    # label_smoothing=0.1 tells the model: "Don't be 100% sure".
    # This prevents it from ignoring the hard classes.
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

# D. Train
callbacks = [
    # Stop training if validation loss doesn't improve for 5 epochs
   # EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),

    # Save the best model automatically in the native Keras format
    ModelCheckpoint('best_lroc_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1),

    # Slow down learning rate if the model gets stuck
    #ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1) # not allowed with cosine LR
]

In [ ]:
STEPS_PER_EPOCH = 300  # reasonable for the dataset
print("🚀 Starting training with Label Smoothing (No Class Weights)...")
history = model.fit(
    train_ds_smooth_1,
    epochs=40,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_ds_smooth,
    callbacks=callbacks,
    verbose=1
    # Note: NO class_weights here!
)
print("✅ Training Complete.")

### 4. Results 

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(15, 5))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

plot_history(history)

#### 4a. For classification report

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Collect labels and predictions
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)   # get class index
    labs = np.argmax(labels.numpy(), axis=1) if labels.ndim > 1 else labels.numpy()

    y_true.extend(labs)
    y_pred.extend(preds)

In [ ]:
class_names = ["ejecta", "oldcrater", "none"]

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# --- Step 1: Normalize confusion matrix row-wise (% per true class) ---
cm_percent = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] * 100

# --- Step 2: Create combined labels ---
labels = np.array([
    [f"{count}\n({percent:.1f}%)" for count, percent in zip(row_counts, row_perc)]
    for row_counts, row_perc in zip(cm, cm_percent)
])

# --- Step 3: Plot normalized heatmap with annotated combined labels ---
plt.figure(figsize=(7,5))
sns.heatmap(
    cm_percent,                # Color scaling uses percentage
    annot=labels,              # Combined count + percentage
    fmt="",                    # Disable default number formatting
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Percentage (%)'}
)

plt.xlabel("Predicted", fontsize=12)
plt.ylabel("True", fontsize=12)
plt.title("Confusion Matrix (Counts + Percentages)", fontsize=14)
plt.tight_layout()
plt.show()
